<a href="https://colab.research.google.com/github/sammysingle/Thesis/blob/main/ColabWorkspaces/preliminary_results.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# VGG-16

*Original Dataset*

In [ ]:
# Import os library to navigate directories
import os
# Import tensorflow functions for model building
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Dense, Input, GlobalAveragePooling2D
from tensorflow.keras import Model
from tensorflow.keras.models import save_model
# Import tensorflow functions for training
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.data import AUTOTUNE as autotune
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow import cast
# Import tensorflow functions for metrics
from tensorflow.keras.metrics import Accuracy, Precision, Recall

# ------------------------------ Build VGG-16 ------------------------------ #
# Create base VGG-16 model:
  # 224x224x3 input size
  # Only feature extraction layers - no classification layers
  # Feature extraction layer weights trained on ImageNet dataset
base_model = VGG16(input_shape = (224, 224, 3),
                         include_top = False,
                         weights = 'imagenet'
                         )
# Freeze learning on feature extraction layers
base_model.trainable = False

# Create classification layers of VGG16
# Global average pooling
global_average_pooling = GlobalAveragePooling2D()
# Fully connected layer 1: 4096 neurons ReLU activation function
fc1 = Dense(units = 4096, activation = 'relu')
# Fully connected layer 2: 4096 neurons ReLU activation function
fc2 = Dense(units = 4096, activation = 'relu')
# Classification layer: softmax layer with 8 neurons for class labels
classifier = Dense(units = 8, activation = 'softmax')

# Connect input layer, base model and output layers
inputs = Input(shape = (224, 224, 3)) # Input layer: 224x224 RGB images
model = base_model(inputs)            # Feature extraction layers of VGG-16
model = global_average_pooling(model) # Global average pooling layer
model = fc1(model)                    # Fully connected layer: 4096 neuron ReLU
model = fc2(model)                    # Fully connected layer: 4096 neuron ReLU
outputs = classifier(model)           # Fully connected layer: 8 neuron softmax activation for classification

# Build VGG-16 model
vgg16 = Model(inputs, outputs, name = 'VGG-16')

# ------------------------------ Train VGG-16 ------------------------------ #
# RGB value normalization to improve learning rate - RGB values within activation function range
def process(image, label):
    image = cast(image/255.0, 'float32')
    return image, label

# Define training parameters
batch_size = 4   # Batch size for gradient learning
img_height = 224 # Image pixel height
img_width = 224  # Image pixel width
epochs = 100     # 100 rounds of training

# Define training and validation directory paths
train_dir = '/content/drive/MyDrive/Colab Notebooks/Model Training and Evaluation/Original Training Dataset'
validation_dir = '/content/drive/MyDrive/Colab Notebooks/Model Training and Evaluation/Validation Dataset'

# Create training dataset
train_ds = image_dataset_from_directory(
    train_dir,
    validation_split = 0,
    seed = 123,
    image_size = (img_height, img_width),
    batch_size = batch_size,
    label_mode = 'categorical'
)
# Normalize training dataset pixel values to improve learning performance
train_ds = train_ds.map(process)
# Prefetch training dataset to improve computational speed
train_ds = train_ds.cache().prefetch(buffer_size = autotune)

# Define validation dataset
val_ds = image_dataset_from_directory(
    validation_dir,
    validation_split = 0,
    seed = 123,
    image_size = (img_height, img_width),
    batch_size = batch_size,
    label_mode = 'categorical'
)
# Normalize validation pixel values to improve learning performance
val_ds = val_ds.map(process)
# Prefetch validation dataset to improve computation speed
val_ds = val_ds.cache().prefetch(buffer_size = autotune)

# Compile model:
  # Adam algorithm optimization
  # Categorical crossentropy loss function for multilabel classification
  # Metrics: accuracy, precision, recall
vgg16.compile(
    optimizer = 'adam',
    loss = 'categorical_crossentropy',
    metrics = ['accuracy', Precision(), Recall()]
)

# Define checkpoint to save model after every epoch
model_checkpoint = ModelCheckpoint('/content/drive/MyDrive/Colab Notebooks/Model Training and Evaluation/Models/VGG-16/Original Training/vgg16-original-{epoch:02d}-{val_accuracy:.4f}.hdf5', save_best_only = False, save_weights_only = False, mode= 'auto')

# Train model
vgg16.fit(
    train_ds,
    validation_data = val_ds,
    epochs = epochs,
    verbose = 1,
    callbacks = [model_checkpoint]
)

Found 2433 files belonging to 8 classes.
Found 695 files belonging to 8 classes.
Epoch 1/100
609/609 [==============================] - 11s 13ms/step - loss: 1.2654 - accuracy: 0.4994 - precision_10: 0.7347 - recall_10: 0.3300 - val_loss: 0.9714 - val_accuracy: 0.6173 - val_precision_10: 0.7600 - val_recall_10: 0.4647
Epoch 2/100
609/609 [==============================] - 7s 11ms/step - loss: 0.9320 - accuracy: 0.6276 - precision_10: 0.7845 - recall_10: 0.4817 - val_loss: 1.0274 - val_accuracy: 0.6014 - val_precision_10: 0.7049 - val_recall_10: 0.5223
Epoch 3/100
609/609 [==============================] - 7s 11ms/step - loss: 0.7974 - accuracy: 0.6917 - precision_10: 0.7977 - recall_10: 0.5882 - val_loss: 0.9034 - val_accuracy: 0.6518 - val_precision_10: 0.7256 - val_recall_10: 0.5784
Epoch 4/100
609/609 [==============================] - 7s 12ms/step - loss: 0.6800 - accuracy: 0.7485 - precision_10: 0.8243 - recall_10: 0.6691 - val_loss: 0.7964 - val_accuracy: 0.7036 - val_precision_1

*Augmented Dataset*

In [ ]:
# Import os library to navigate directories
import os
# Import tensorflow functions for model building
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Dense, Input, GlobalAveragePooling2D
from tensorflow.keras import Model
from tensorflow.keras.models import save_model
# Import tensorflow functions for training
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.data import AUTOTUNE as autotune
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow import cast
# Import tensorflow functions for metrics
from tensorflow.keras.metrics import Accuracy, Precision, Recall

# ------------------------------ Build VGG-16 ------------------------------ #
# Create base VGG-16 model:
  # 224x224x3 input size
  # Only feature extraction layers - no classification layers
  # Feature extraction layer weights trained on ImageNet dataset
base_model = VGG16(input_shape = (224, 224, 3),
                         include_top = False,
                         weights = 'imagenet'
                         )
# Freeze learning on feature extraction layers
base_model.trainable = False

# Create classification layers of VGG16
# Global average pooling
global_average_pooling = GlobalAveragePooling2D()
# Fully connected layer 1: 4096 neurons ReLU activation function
fc1 = Dense(units = 4096, activation = 'relu')
# Fully connected layer 2: 4096 neurons ReLU activation function
fc2 = Dense(units = 4096, activation = 'relu')
# Classification layer: softmax layer with 8 neurons for class labels
classifier = Dense(units = 8, activation = 'softmax')

# Connect input layer, base model and output layers
inputs = Input(shape = (224, 224, 3)) # Input layer: 224x224 RGB images
model = base_model(inputs)            # Feature extraction layers of VGG-16
model = global_average_pooling(model) # Global average pooling layer
model = fc1(model)                    # Fully connected layer: 4096 neuron ReLU
model = fc2(model)                    # Fully connected layer: 4096 neuron ReLU
outputs = classifier(model)           # Fully connected layer: 8 neuron softmax activation for classification

# Build VGG-16 model
vgg16 = Model(inputs, outputs, name = 'VGG-16')

# ------------------------------ Train VGG-16 ------------------------------ #
# RGB value normalization to improve learning rate - RGB values within activation function range
def process(image, label):
    image = cast(image/255.0, 'float32')
    return image, label

# Define training parameters
batch_size = 4   # Batch size for gradient learning
img_height = 224 # Image pixel height
img_width = 224  # Image pixel width
epochs = 100     # 100 rounds of training

# Define training and validation directory paths
train_dir = '/content/drive/MyDrive/Colab Notebooks/Model Training and Evaluation/Augmented Training Dataset'
validation_dir = '/content/drive/MyDrive/Colab Notebooks/Model Training and Evaluation/Validation Dataset'

# Create training dataset
train_ds = image_dataset_from_directory(
    train_dir,
    validation_split = 0,
    seed = 123,
    image_size = (img_height, img_width),
    batch_size = batch_size,
    label_mode = 'categorical'
)
# Normalize training dataset pixel values to improve learning performance
train_ds = train_ds.map(process)
# Prefetch training dataset to improve computational speed
train_ds = train_ds.cache().prefetch(buffer_size = autotune)

# Define validation dataset
val_ds = image_dataset_from_directory(
    validation_dir,
    validation_split = 0,
    seed = 123,
    image_size = (img_height, img_width),
    batch_size = batch_size,
    label_mode = 'categorical'
)
# Normalize validation pixel values to improve learning performance
val_ds = val_ds.map(process)
# Prefetch validation dataset to improve computation speed
val_ds = val_ds.cache().prefetch(buffer_size = autotune)

# Compile model:
  # Adam algorithm optimization
  # Categorical crossentropy loss function for multilabel classification
  # Metrics: accuracy, precision, recall
vgg16.compile(
    optimizer = 'adam',
    loss = 'categorical_crossentropy',
    metrics = ['accuracy', Precision(), Recall()]
)

# Define checkpoint to save model after every epoch
model_checkpoint = ModelCheckpoint('/content/drive/MyDrive/Colab Notebooks/Model Training and Evaluation/Models/VGG-16/Augmented Training/vgg16-augmented-{epoch:02d}-{val_accuracy:.4f}.hdf5', save_best_only = False, save_weights_only = False, mode= 'auto')

# Train model
vgg16.fit(
    train_ds,
    validation_data = val_ds,
    epochs = epochs,
    verbose = 1,
    callbacks = [model_checkpoint]
)

Found 10083 files belonging to 8 classes.
Found 695 files belonging to 8 classes.
Epoch 1/100
2521/2521 [==============================] - 25s 10ms/step - loss: 1.1144 - accuracy: 0.5824 - precision_11: 0.7711 - recall_11: 0.4243 - val_loss: 0.7165 - val_accuracy: 0.7612 - val_precision_11: 0.8516 - val_recall_11: 0.6360
Epoch 2/100
2521/2521 [==============================] - 23s 9ms/step - loss: 0.7845 - accuracy: 0.7207 - precision_11: 0.8125 - recall_11: 0.6208 - val_loss: 0.6267 - val_accuracy: 0.7885 - val_precision_11: 0.8591 - val_recall_11: 0.7194
Epoch 3/100
2521/2521 [==============================] - 23s 9ms/step - loss: 0.6450 - accuracy: 0.7638 - precision_11: 0.8351 - recall_11: 0.6938 - val_loss: 0.6679 - val_accuracy: 0.7669 - val_precision_11: 0.8111 - val_recall_11: 0.7353
Epoch 4/100
2521/2521 [==============================] - 23s 9ms/step - loss: 0.5538 - accuracy: 0.7937 - precision_11: 0.8514 - recall_11: 0.7375 - val_loss: 0.6524 - val_accuracy: 0.7698 - val_pr

# ResNet-50

*Original Dataset*

In [ ]:
# Import os library to navigate directories
import os
# Import tensorflow functions for model building
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import Dense, Input, GlobalAveragePooling2D
from tensorflow.keras import Model
from tensorflow.keras.models import save_model
# Import tensorflow functions for training
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.data import AUTOTUNE as autotune
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow import cast
# Import tensorflow functions for metrics
from tensorflow.keras.metrics import Accuracy, Precision, Recall

# ----------------------------- Build ResNet-50 ---------------------------- #
# Create base ResNet-50 model:
  # 224x224x3 input size
  # Only feature extraction layers - no classification layers
  # Feature extraction layer weights trained on ImageNet dataset
base_model = ResNet50(input_shape = (224, 224, 3),
                         include_top = False,
                         weights = 'imagenet'
                         )
# Freeze learning on feature extraction layers
base_model.trainable = False

# Create classification layers of ResNet-50
global_average_pool = GlobalAveragePooling2D()        # Global average pooling
classifier = Dense(units = 8, activation = 'softmax') # Softmax classifier

# Connect input layer, base model and output layers
inputs = Input(shape = (224, 224, 3)) # Input layer: 224x224 RGB images
model = base_model(inputs)            # Feature extraction layers of ResNet-50
model = global_average_pool(model)    # Global average pooling layer
outputs = classifier(model)           # Fully connected layer: 8 neuron softmax activation for classification

# Build res_net_50 model
res_net_50 = Model(inputs, outputs, name = 'ResNet-50')

# ----------------------------- Train ResNet-50 ---------------------------- #
# RGB value normalization to improve learning rate - RGB values within activation function range
def process(image, label):
    image = cast(image/255.0, 'float32')
    return image, label

# Define training parameters
batch_size = 4   # Batch size for gradient learning
img_height = 224 # Image pixel height
img_width = 224  # Image pixel width
epochs = 100     # 100 rounds of training

# Define training and validation directory paths
train_dir = '/content/drive/MyDrive/Colab Notebooks/Model Training and Evaluation/Original Training Dataset'
validation_dir = '/content/drive/MyDrive/Colab Notebooks/Model Training and Evaluation/Validation Dataset'

# Create training dataset
train_ds = image_dataset_from_directory(
    train_dir,
    validation_split = 0,
    seed = 123,
    image_size = (img_height, img_width),
    batch_size = batch_size,
    label_mode = 'categorical'
)
# Normalize training dataset pixel values to improve learning performance
train_ds = train_ds.map(process)
# Prefetch training dataset to improve computational speed
train_ds = train_ds.cache().prefetch(buffer_size = autotune)

# Define validation dataset
val_ds = image_dataset_from_directory(
    validation_dir,
    validation_split = 0,
    seed = 123,
    image_size = (img_height, img_width),
    batch_size = batch_size,
    label_mode = 'categorical'
)
# Normalize validation pixel values to improve learning performance
val_ds = val_ds.map(process)
# Prefetch validation dataset to improve computation speed
val_ds = val_ds.cache().prefetch(buffer_size = autotune)

# Compile model:
  # Adam algorithm optimization
  # Categorical crossentropy loss function for multilabel classification
  # Metrics: accuracy, precision, recall
res_net_50.compile(
    optimizer = 'adam',
    loss = 'categorical_crossentropy',
    metrics = ['accuracy', Precision(), Recall()]
)

# Define checkpoint to save model after every epoch
model_checkpoint = ModelCheckpoint('/content/drive/MyDrive/Colab Notebooks/Model Training and Evaluation/Models/ResNet-50/Original Training/resnet50-original-{epoch:02d}-{val_accuracy:.4f}.hdf5', save_best_only = False, save_weights_only = False, mode= 'auto')

# Train model
res_net_50.fit(
    train_ds,
    validation_data = val_ds,
    epochs = epochs,
    verbose = 1,
    callbacks = [model_checkpoint]
)

94781440/94765736 [==============================] - 0s 0us/step
Found 2433 files belonging to 8 classes.
Found 695 files belonging to 8 classes.
Epoch 1/100
609/609 [==============================] - 87s 122ms/step - loss: 2.0411 - accuracy: 0.1928 - precision: 0.5938 - recall: 0.0078 - val_loss: 1.9465 - val_accuracy: 0.1957 - val_precision: 0.4737 - val_recall: 0.0129
Epoch 2/100
609/609 [==============================] - 9s 14ms/step - loss: 1.9328 - accuracy: 0.2565 - precision: 0.6528 - recall: 0.0193 - val_loss: 1.8805 - val_accuracy: 0.2561 - val_precision: 0.5600 - val_recall: 0.0201
Epoch 3/100
609/609 [==============================] - 9s 14ms/step - loss: 1.8737 - accuracy: 0.2926 - precision: 0.7297 - recall: 0.0333 - val_loss: 1.8348 - val_accuracy: 0.2835 - val_precision: 0.6250 - val_recall: 0.0288
Epoch 4/100
609/609 [==============================] - 9s 14ms/step - loss: 1.8283 - accuracy: 0.3173 - precision: 0.7895 - recall: 0.0493 - val_loss: 1.7993 - val_accuracy: 

*Augmented Dataset*

In [ ]:
# Import os library to navigate directories
import os
# Import tensorflow functions for model building
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import Dense, Input, GlobalAveragePooling2D
from tensorflow.keras import Model
from tensorflow.keras.models import save_model
# Import tensorflow functions for training
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.data import AUTOTUNE as autotune
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow import cast
# Import tensorflow functions for metrics
from tensorflow.keras.metrics import Accuracy, Precision, Recall

# ----------------------------- Build ResNet-50 ---------------------------- #
# Create base ResNet-50 model:
  # 224x224x3 input size
  # Only feature extraction layers - no classification layers
  # Feature extraction layer weights trained on ImageNet dataset
base_model = ResNet50(input_shape = (224, 224, 3),
                         include_top = False,
                         weights = 'imagenet'
                         )
# Freeze learning on feature extraction layers
base_model.trainable = False

# Create classification layers of ResNet-50
global_average_pool = GlobalAveragePooling2D()        # Global average pooling
classifier = Dense(units = 8, activation = 'softmax') # Softmax classifier

# Connect input layer, base model and output layers
inputs = Input(shape = (224, 224, 3)) # Input layer: 224x224 RGB images
model = base_model(inputs)            # Feature extraction layers of ResNet-50
model = global_average_pool(model)    # Global average pooling layer
outputs = classifier(model)           # Fully connected layer: 8 neuron softmax activation for classification

# Build res_net_50 model
res_net_50 = Model(inputs, outputs, name = 'ResNet-50')

# ----------------------------- Train ResNet-50 ---------------------------- #
# RGB value normalization to improve learning rate - RGB values within activation function range
def process(image, label):
    image = cast(image/255.0, 'float32')
    return image, label

# Define training parameters
batch_size = 4   # Batch size for gradient learning
img_height = 224 # Image pixel height
img_width = 224  # Image pixel width
epochs = 100     # 100 rounds of training

# Define training and validation directory paths
train_dir = '/content/drive/MyDrive/Colab Notebooks/Model Training and Evaluation/Augmented Training Dataset'
validation_dir = '/content/drive/MyDrive/Colab Notebooks/Model Training and Evaluation/Validation Dataset'

# Create training dataset
train_ds = image_dataset_from_directory(
    train_dir,
    validation_split = 0,
    seed = 123,
    image_size = (img_height, img_width),
    batch_size = batch_size,
    label_mode = 'categorical'
)
# Normalize training dataset pixel values to improve learning performance
train_ds = train_ds.map(process)
# Prefetch training dataset to improve computational speed
train_ds = train_ds.cache().prefetch(buffer_size = autotune)

# Define validation dataset
val_ds = image_dataset_from_directory(
    validation_dir,
    validation_split = 0,
    seed = 123,
    image_size = (img_height, img_width),
    batch_size = batch_size,
    label_mode = 'categorical'
)
# Normalize validation pixel values to improve learning performance
val_ds = val_ds.map(process)
# Prefetch validation dataset to improve computation speed
val_ds = val_ds.cache().prefetch(buffer_size = autotune)

# Compile model:
  # Adam algorithm optimization
  # Categorical crossentropy loss function for multilabel classification
  # Metrics: accuracy, precision, recall
res_net_50.compile(
    optimizer = 'adam',
    loss = 'categorical_crossentropy',
    metrics = ['accuracy', Precision(), Recall()]
)

# Define checkpoint to save model after every epoch
model_checkpoint = ModelCheckpoint('/content/drive/MyDrive/Colab Notebooks/Model Training and Evaluation/Models/ResNet-50/Augmented Training/resnet50-augmented-{epoch:02d}-{val_accuracy:.4f}.hdf5', save_best_only = False, save_weights_only = False, mode= 'auto')

# Train model
res_net_50.fit(
    train_ds,
    validation_data = val_ds,
    epochs = epochs,
    verbose = 1,
    callbacks = [model_checkpoint]
)

Found 10083 files belonging to 8 classes.
Found 695 files belonging to 8 classes.
Epoch 1/100
2521/2521 [==============================] - 218s 85ms/step - loss: 1.9453 - accuracy: 0.2497 - precision_1: 0.7025 - recall_1: 0.0227 - val_loss: 1.7940 - val_accuracy: 0.3741 - val_precision_1: 0.6071 - val_recall_1: 0.0245
Epoch 2/100
2521/2521 [==============================] - 29s 12ms/step - loss: 1.8096 - accuracy: 0.3252 - precision_1: 0.7958 - recall_1: 0.0646 - val_loss: 1.7074 - val_accuracy: 0.4129 - val_precision_1: 0.7143 - val_recall_1: 0.0576
Epoch 3/100
2521/2521 [==============================] - 30s 12ms/step - loss: 1.7412 - accuracy: 0.3513 - precision_1: 0.8107 - recall_1: 0.0934 - val_loss: 1.6534 - val_accuracy: 0.4259 - val_precision_1: 0.8000 - val_recall_1: 0.0863
Epoch 4/100
2521/2521 [==============================] - 30s 12ms/step - loss: 1.6955 - accuracy: 0.3680 - precision_1: 0.8150 - recall_1: 0.1123 - val_loss: 1.6143 - val_accuracy: 0.4432 - val_precision_1:

# DenseNet121

*Original Dataset*

In [ ]:
# Import os library to navigate directories
import os
# Import tensorflow functions for model building
from tensorflow.keras.applications.densenet import DenseNet121
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Input
from tensorflow.keras import Model
from tensorflow.keras.models import save_model
# Import tensorflow functions for training
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.data import AUTOTUNE as autotune
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow import cast
# Import tensorflow functions for metrics
from tensorflow.keras.metrics import Accuracy, Precision, Recall

# --------------------------- Build DenseNet121 ---------------------------- #
# Create base DenseNet121 model:
  # 224x224x3 input size
  # Only feature extraction layers - no classification layers
  # Feature extraction layer weights trained on ImageNet dataset
base_model = DenseNet121(input_shape = (224, 224, 3),
                         include_top = False,
                         weights = 'imagenet'
                         )
# Freeze learning on feature extraction layers
base_model.trainable = False

# Create classification layers of DenseNet121
# 7x7 global average pooling layer
global_average_pooling = GlobalAveragePooling2D()
# Fully connected layer using softmax activation function for 8 labels
classifier = Dense(units = 8, activation = 'softmax')

# Connect input layer, base model and output classifier
inputs = Input(shape = (224, 224, 3)) # Input layer: 224x224 RGB images
model = base_model(inputs)            # Feature extraction layers of DenseNet121
model = global_average_pooling(model) # Fully connected layer: 7x7 global average pooling
outputs = classifier(model)           # Fully connected layer: 8 neuron softmax activation for classification

# Build DenseNet121 model
dense_net_121 = Model(inputs, outputs, name = 'DenseNet121')

# --------------------------- Train DenseNet121 ---------------------------- #
# RGB value normalization to improve learning rate - RGB values within activation function range
def process(image, label):
    image = cast(image/255.0, 'float32')
    return image, label

# Define training parameters
batch_size = 4   # Batch size for gradient learning
img_height = 224 # Image pixel height
img_width = 224  # Image pixel width
epochs = 100     # 100 rounds of training

# Define training and validation directory paths
train_dir = '/content/drive/MyDrive/Colab Notebooks/Model Training and Evaluation/Original Training Dataset'
validation_dir = '/content/drive/MyDrive/Colab Notebooks/Model Training and Evaluation/Validation Dataset'

# Create training dataset
train_ds = image_dataset_from_directory(
    train_dir,
    validation_split = 0,
    seed = 123,
    image_size = (img_height, img_width),
    batch_size = batch_size,
    label_mode = 'categorical'
)
# Normalize training dataset pixel values to improve learning performance
train_ds = train_ds.map(process)
# Prefetch training dataset to improve computational speed
train_ds = train_ds.cache().prefetch(buffer_size = autotune)

# Define validation dataset
val_ds = image_dataset_from_directory(
    validation_dir,
    validation_split = 0,
    seed = 123,
    image_size = (img_height, img_width),
    batch_size = batch_size,
    label_mode = 'categorical'
)
# Normalize validation pixel values to improve learning performance
val_ds = val_ds.map(process)
# Prefetch validation dataset to improve computation speed
val_ds = val_ds.cache().prefetch(buffer_size = autotune)

# Compile model:
  # Adam algorithm optimization
  # Categorical crossentropy loss function for multilabel classification
  # Metrics: accuracy, precision, recall
dense_net_121.compile(
    optimizer = 'adam',
    loss = 'categorical_crossentropy',
    metrics = ['accuracy', Precision(), Recall()]
)

# Define checkpoint to save model after every epoch
model_checkpoint = ModelCheckpoint('/content/drive/MyDrive/Colab Notebooks/Model Training and Evaluation/Models/DenseNet121/Original Training/densenet121-original-{epoch:02d}-{val_accuracy:.4f}.hdf5', save_best_only = False, save_weights_only = False, mode= 'auto')

# Train model
dense_net_121.fit(
    train_ds,
    validation_data = val_ds,
    epochs = epochs,
    verbose = 1,
    callbacks = [model_checkpoint]
)

29097984/29084464 [==============================] - 0s 0us/step
Found 2433 files belonging to 8 classes.
Found 695 files belonging to 8 classes.
Epoch 1/100
609/609 [==============================] - 30s 33ms/step - loss: 0.8954 - accuracy: 0.6860 - precision_2: 0.8290 - recall_2: 0.5479 - val_loss: 0.5789 - val_accuracy: 0.7784 - val_precision_2: 0.8517 - val_recall_2: 0.7108
Epoch 2/100
609/609 [==============================] - 16s 26ms/step - loss: 0.4733 - accuracy: 0.8422 - precision_2: 0.8962 - recall_2: 0.7846 - val_loss: 0.4954 - val_accuracy: 0.8086 - val_precision_2: 0.8645 - val_recall_2: 0.7712
Epoch 3/100
609/609 [==============================] - 16s 26ms/step - loss: 0.3669 - accuracy: 0.8788 - precision_2: 0.9209 - recall_2: 0.8376 - val_loss: 0.4647 - val_accuracy: 0.8230 - val_precision_2: 0.8648 - val_recall_2: 0.7914
Epoch 4/100
609/609 [==============================] - 16s 26ms/step - loss: 0.3032 - accuracy: 0.9067 - precision_2: 0.9388 - recall_2: 0.8697 - val

*Augmented Dataset*

In [ ]:
# Import os library to navigate directories
import os
# Import tensorflow functions for model building
from tensorflow.keras.applications.densenet import DenseNet121
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Input
from tensorflow.keras import Model
from tensorflow.keras.models import save_model
# Import tensorflow functions for training
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.data import AUTOTUNE as autotune
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow import cast
# Import tensorflow functions for metrics
from tensorflow.keras.metrics import Accuracy, Precision, Recall

# --------------------------- Build DenseNet121 ---------------------------- #
# Create base DenseNet121 model:
  # 224x224x3 input size
  # Only feature extraction layers - no classification layers
  # Feature extraction layer weights trained on ImageNet dataset
base_model = DenseNet121(input_shape = (224, 224, 3),
                         include_top = False,
                         weights = 'imagenet'
                         )
# Freeze learning on feature extraction layers
base_model.trainable = False

# Create classification layers of DenseNet121
# 7x7 global average pooling layer
global_average_pooling = GlobalAveragePooling2D()
# Fully connected layer using softmax activation function for 8 labels
classifier = Dense(units = 8, activation = 'softmax')

# Connect input layer, base model and output classifier
inputs = Input(shape = (224, 224, 3)) # Input layer: 224x224 RGB images
model = base_model(inputs)            # Feature extraction layers of DenseNet121
model = global_average_pooling(model) # Fully connected layer: 7x7 global average pooling
outputs = classifier(model)           # Fully connected layer: 8 neuron softmax activation for classification

# Build DenseNet121 model
dense_net_121 = Model(inputs, outputs, name = 'DenseNet121')

# --------------------------- Train DenseNet121 ---------------------------- #
# RGB value normalization to improve learning rate - RGB values within activation function range
def process(image, label):
    image = cast(image/255.0, 'float32')
    return image, label

# Define training parameters
batch_size = 4   # Batch size for gradient learning
img_height = 224 # Image pixel height
img_width = 224  # Image pixel width
epochs = 100     # 100 rounds of training

# Define training and validation directory paths
train_dir = '/content/drive/MyDrive/Colab Notebooks/Model Training and Evaluation/Augmented Training Dataset'
validation_dir = '/content/drive/MyDrive/Colab Notebooks/Model Training and Evaluation/Validation Dataset'

# Create training dataset
train_ds = image_dataset_from_directory(
    train_dir,
    validation_split = 0,
    seed = 123,
    image_size = (img_height, img_width),
    batch_size = batch_size,
    label_mode = 'categorical'
)
# Normalize training dataset pixel values to improve learning performance
train_ds = train_ds.map(process)
# Prefetch training dataset to improve computational speed
train_ds = train_ds.cache().prefetch(buffer_size = autotune)

# Define validation dataset
val_ds = image_dataset_from_directory(
    validation_dir,
    validation_split = 0,
    seed = 123,
    image_size = (img_height, img_width),
    batch_size = batch_size,
    label_mode = 'categorical'
)
# Normalize validation pixel values to improve learning performance
val_ds = val_ds.map(process)
# Prefetch validation dataset to improve computation speed
val_ds = val_ds.cache().prefetch(buffer_size = autotune)

# Compile model:
  # Adam algorithm optimization
  # Categorical crossentropy loss function for multilabel classification
  # Metrics: accuracy, precision, recall
dense_net_121.compile(
    optimizer = 'adam',
    loss = 'categorical_crossentropy',
    metrics = ['accuracy', Precision(), Recall()]
)

# Define checkpoint to save model after every epoch
model_checkpoint = ModelCheckpoint('/content/drive/MyDrive/Colab Notebooks/Model Training and Evaluation/Models/DenseNet121/Augmented Training/densenet121-augmented-{epoch:02d}-{val_accuracy:.4f}.hdf5', save_best_only = False, save_weights_only = False, mode= 'auto')

# Train model
dense_net_121.fit(
    train_ds,
    validation_data = val_ds,
    epochs = epochs,
    verbose = 1,
    callbacks = [model_checkpoint]
)

Found 10083 files belonging to 8 classes.
Found 695 files belonging to 8 classes.
Epoch 1/100
2521/2521 [==============================] - 67s 24ms/step - loss: 0.7062 - accuracy: 0.7531 - precision_3: 0.8499 - recall_3: 0.6653 - val_loss: 0.4663 - val_accuracy: 0.8432 - val_precision_3: 0.8700 - val_recall_3: 0.8086
Epoch 2/100
2521/2521 [==============================] - 53s 21ms/step - loss: 0.4327 - accuracy: 0.8453 - precision_3: 0.8872 - recall_3: 0.8048 - val_loss: 0.4563 - val_accuracy: 0.8518 - val_precision_3: 0.8851 - val_recall_3: 0.8317
Epoch 3/100
2521/2521 [==============================] - 53s 21ms/step - loss: 0.3638 - accuracy: 0.8704 - precision_3: 0.9005 - recall_3: 0.8401 - val_loss: 0.4695 - val_accuracy: 0.8532 - val_precision_3: 0.8826 - val_recall_3: 0.8331
Epoch 4/100
2521/2521 [==============================] - 54s 21ms/step - loss: 0.3246 - accuracy: 0.8816 - precision_3: 0.9067 - recall_3: 0.8568 - val_loss: 0.4875 - val_accuracy: 0.8504 - val_precision_3: 

# Inception V3

*Original Dataset*

In [ ]:
# Import os library to navigate directories
import os
# Import tensorflow functions for model building
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.layers import Dense, Input, GlobalAveragePooling2D
from tensorflow.keras import Model
from tensorflow.keras.models import save_model
# Import tensorflow functions for training
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.data import AUTOTUNE as autotune
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow import cast
# Import tensorflow functions for metrics
from tensorflow.keras.metrics import Accuracy, Precision, Recall

# ---------------------------- Build InceptionV3 --------------------------- #
# Create base InceptionV3 model:
  # 224x224x3 input size
  # Only feature extraction layers - no classification layers
  # Feature extraction layer weights trained on ImageNet dataset
base_model = InceptionV3(input_shape = (224, 224, 3),
                         include_top = False,
                         weights = 'imagenet'
                         )
# Freeze learning on feature extraction layers
base_model.trainable = False

# Create classification layers of InceptionV3
global_average_pool = GlobalAveragePooling2D()        # global average pooling
fc1 = Dense(units = 2048, activation = 'relu')        # 2048 neuron fully connected layer with ReLU activation
classifier = Dense(units = 8, activation = 'softmax') # Softmax classifier

# Connect input layer, base model and output layers
inputs = Input(shape = (224, 224, 3)) # Input layer: 224x224 RGB images
model = base_model(inputs)            # Feature extraction layers of InceptionV3
model = global_average_pool(model)    # Global average pooling layer
model = fc1(model)                    # Fully connected layer: 2048 neuron ReLU
outputs = classifier(model)           # Fully connected layer: 8 neuron softmax activation for classification

# Build inception_v3 model
inception_v3 = Model(inputs, outputs, name = 'InceptionV3')

# ---------------------------- Train InceptionV3 -------------------------- #
# RGB value normalization to improve learning rate - RGB values within activation function range
def process(image, label):
    image = cast(image/255.0, 'float32')
    return image, label

# Define training parameters
batch_size = 4   # Batch size for gradient learning
img_height = 224 # Image pixel height
img_width = 224  # Image pixel width
epochs = 100     # 100 rounds of training

# Define training and validation directory paths
train_dir = '/content/drive/MyDrive/Colab Notebooks/Model Training and Evaluation/Original Training Dataset'
validation_dir = '/content/drive/MyDrive/Colab Notebooks/Model Training and Evaluation/Validation Dataset'

# Create training dataset
train_ds = image_dataset_from_directory(
    train_dir,
    validation_split = 0,
    seed = 123,
    image_size = (img_height, img_width),
    batch_size = batch_size,
    label_mode = 'categorical'
)
# Normalize training dataset pixel values to improve learning performance
train_ds = train_ds.map(process)
# Prefetch training dataset to improve computational speed
train_ds = train_ds.cache().prefetch(buffer_size = autotune)

# Define validation dataset
val_ds = image_dataset_from_directory(
    validation_dir,
    validation_split = 0,
    seed = 123,
    image_size = (img_height, img_width),
    batch_size = batch_size,
    label_mode = 'categorical'
)
# Normalize validation pixel values to improve learning performance
val_ds = val_ds.map(process)
# Prefetch validation dataset to improve computation speed
val_ds = val_ds.cache().prefetch(buffer_size = autotune)

# Compile model:
  # Adam algorithm optimization
  # Categorical crossentropy loss function for multilabel classification
  # Metrics: accuracy, precision, recall
inception_v3.compile(
    optimizer = 'adam',
    loss = 'categorical_crossentropy',
    metrics = ['accuracy', Precision(), Recall()]
)

# Define checkpoint to save model after every epoch
model_checkpoint = ModelCheckpoint('/content/drive/MyDrive/Colab Notebooks/Model Training and Evaluation/Models/InceptionV3/Original Training/inceptionv3-original-{epoch:02d}-{val_accuracy:.4f}.hdf5', save_best_only = False, save_weights_only = False, mode= 'auto')

# Train model
inception_v3.fit(
    train_ds,
    validation_data = val_ds,
    epochs = epochs,
    verbose = 1,
    callbacks = [model_checkpoint]
)

87924736/87910968 [==============================] - 0s 0us/step
Found 2433 files belonging to 8 classes.
Found 695 files belonging to 8 classes.
Epoch 1/100
609/609 [==============================] - 23s 28ms/step - loss: 1.0161 - accuracy: 0.7065 - precision_4: 0.7648 - recall_4: 0.6457 - val_loss: 0.6388 - val_accuracy: 0.7568 - val_precision_4: 0.8134 - val_recall_4: 0.6964
Epoch 2/100
609/609 [==============================] - 14s 22ms/step - loss: 0.4463 - accuracy: 0.8376 - precision_4: 0.8767 - recall_4: 0.8093 - val_loss: 0.5755 - val_accuracy: 0.7986 - val_precision_4: 0.8320 - val_recall_4: 0.7554
Epoch 3/100
609/609 [==============================] - 14s 22ms/step - loss: 0.2924 - accuracy: 0.8948 - precision_4: 0.9154 - recall_4: 0.8759 - val_loss: 0.6501 - val_accuracy: 0.7813 - val_precision_4: 0.7997 - val_recall_4: 0.7583
Epoch 4/100
609/609 [==============================] - 14s 22ms/step - loss: 0.2270 - accuracy: 0.9207 - precision_4: 0.9308 - recall_4: 0.9116 - val

*Augmented Dataset*

In [1]:
# Import os library to navigate directories
import os
# Import tensorflow functions for model building
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.layers import Dense, Input, GlobalAveragePooling2D
from tensorflow.keras import Model
from tensorflow.keras.models import save_model
# Import tensorflow functions for training
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.data import AUTOTUNE as autotune
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow import cast
# Import tensorflow functions for metrics
from tensorflow.keras.metrics import Accuracy, Precision, Recall

# ---------------------------- Build InceptionV3 --------------------------- #
# Create base InceptionV3 model:
  # 224x224x3 input size
  # Only feature extraction layers - no classification layers
  # Feature extraction layer weights trained on ImageNet dataset
base_model = InceptionV3(input_shape = (224, 224, 3),
                         include_top = False,
                         weights = 'imagenet'
                         )
# Freeze learning on feature extraction layers
base_model.trainable = False

# Create classification layers of InceptionV3
global_average_pool = GlobalAveragePooling2D()        # global average pooling
fc1 = Dense(units = 2048, activation = 'relu')        # 2048 neuron fully connected layer with ReLU activation
classifier = Dense(units = 8, activation = 'softmax') # Softmax classifier

# Connect input layer, base model and output layers
inputs = Input(shape = (224, 224, 3)) # Input layer: 224x224 RGB images
model = base_model(inputs)            # Feature extraction layers of InceptionV3
model = global_average_pool(model)    # Global average pooling layer
model = fc1(model)                    # Fully connected layer: 2048 neuron ReLU
outputs = classifier(model)           # Fully connected layer: 8 neuron softmax activation for classification

# Build inception_v3 model
inception_v3 = Model(inputs, outputs, name = 'InceptionV3')

# ---------------------------- Train InceptionV3 -------------------------- #
# RGB value normalization to improve learning rate - RGB values within activation function range
def process(image, label):
    image = cast(image/255.0, 'float32')
    return image, label

# Define training parameters
batch_size = 4   # Batch size for gradient learning
img_height = 224 # Image pixel height
img_width = 224  # Image pixel width
epochs = 100     # 100 rounds of training

# Define training and validation directory paths
train_dir = '/content/drive/MyDrive/Colab Notebooks/Model Training and Evaluation/Augmented Training Dataset'
validation_dir = '/content/drive/MyDrive/Colab Notebooks/Model Training and Evaluation/Validation Dataset'

# Create training dataset
train_ds = image_dataset_from_directory(
    train_dir,
    validation_split = 0,
    seed = 123,
    image_size = (img_height, img_width),
    batch_size = batch_size,
    label_mode = 'categorical'
)
# Normalize training dataset pixel values to improve learning performance
train_ds = train_ds.map(process)
# Prefetch training dataset to improve computational speed
train_ds = train_ds.cache().prefetch(buffer_size = autotune)

# Define validation dataset
val_ds = image_dataset_from_directory(
    validation_dir,
    validation_split = 0,
    seed = 123,
    image_size = (img_height, img_width),
    batch_size = batch_size,
    label_mode = 'categorical'
)
# Normalize validation pixel values to improve learning performance
val_ds = val_ds.map(process)
# Prefetch validation dataset to improve computation speed
val_ds = val_ds.cache().prefetch(buffer_size = autotune)

# Compile model:
  # Adam algorithm optimization
  # Categorical crossentropy loss function for multilabel classification
  # Metrics: accuracy, precision, recall
inception_v3.compile(
    optimizer = 'adam',
    loss = 'categorical_crossentropy',
    metrics = ['accuracy', Precision(), Recall()]
)

# Define checkpoint to save model after every epoch
model_checkpoint = ModelCheckpoint('/content/drive/MyDrive/Colab Notebooks/Model Training and Evaluation/Models/InceptionV3/Augmented Training/inceptionv3-augmented-{epoch:02d}-{val_accuracy:.4f}.hdf5', save_best_only = False, save_weights_only = False, mode= 'auto')

# Train model
inception_v3.fit(
    train_ds,
    validation_data = val_ds,
    epochs = epochs,
    verbose = 1,
    callbacks = [model_checkpoint]
)

87924736/87910968 [==============================] - 0s 0us/step
Found 10083 files belonging to 8 classes.
Found 695 files belonging to 8 classes.
Epoch 1/100
2521/2521 [==============================] - 218s 81ms/step - loss: 0.7656 - accuracy: 0.7448 - precision: 0.8069 - recall: 0.6855 - val_loss: 0.5305 - val_accuracy: 0.8144 - val_precision: 0.8676 - val_recall: 0.7640
Epoch 2/100
2521/2521 [==============================] - 43s 17ms/step - loss: 0.4166 - accuracy: 0.8490 - precision: 0.8803 - recall: 0.8199 - val_loss: 0.6202 - val_accuracy: 0.8158 - val_precision: 0.8331 - val_recall: 0.7971
Epoch 3/100
2521/2521 [==============================] - 43s 17ms/step - loss: 0.2891 - accuracy: 0.8905 - precision: 0.9068 - recall: 0.8753 - val_loss: 0.9065 - val_accuracy: 0.8058 - val_precision: 0.8197 - val_recall: 0.7914
Epoch 4/100
2521/2521 [==============================] - 43s 17ms/step - loss: 0.2298 - accuracy: 0.9087 - precision: 0.9187 - recall: 0.8995 - val_loss: 0.9712 - va

# MobileNet V2

*Original Dataset*

In [2]:
# Import os library to navigate directories
import os
# Import tensorflow functions for model building
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.layers import Dense, Input, GlobalAveragePooling2D
from tensorflow.keras import Model
from tensorflow.keras.models import save_model
# Import tensorflow functions for training
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.data import AUTOTUNE as autotune
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow import cast
# Import tensorflow functions for metrics
from tensorflow.keras.metrics import Accuracy, Precision, Recall

# ---------------------------- Build MobileNetV2 --------------------------- #
# Create base MobileNetV2 model:
  # 224x224x3 input size
  # Only feature extraction layers - no classification layers
  # Feature extraction layer weights trained on ImageNet dataset
base_model = MobileNetV2(input_shape = (224, 224, 3),
                         include_top = False,
                         weights = 'imagenet'
                         )
# Freeze learning on feature extraction layers
base_model.trainable = False

# Create classification layers of InceptionV3
global_average_pool = GlobalAveragePooling2D()        # global average pooling
classifier = Dense(units = 8, activation = 'softmax') # 1000 neuron fully connected layer with ReLU activation

# Connect input layer, base model and output layers
inputs = Input(shape = (224, 224, 3)) # Input layer: 224x224 RGB images
model = base_model(inputs)            # Feature extraction layers of InceptionV3
model = global_average_pool(model)    # Global average pooling layer
outputs = classifier(model)           # Fully connected layer: 8 neuron softmax activation for classification

# Build inception_v3 model
mobile_net_v2 = Model(inputs, outputs, name = 'MobileNetV2')

# ---------------------------- Train MobileNetV2 -------------------------- #
# RGB value normalization to improve learning rate - RGB values within activation function range
def process(image, label):
    image = cast(image/255.0, 'float32')
    return image, label

# Define training parameters
batch_size = 4   # Batch size for gradient learning
img_height = 224 # Image pixel height
img_width = 224  # Image pixel width
epochs = 100     # 100 rounds of training

# Define training and validation directory paths
train_dir = '/content/drive/MyDrive/Colab Notebooks/Model Training and Evaluation/Original Training Dataset'
validation_dir = '/content/drive/MyDrive/Colab Notebooks/Model Training and Evaluation/Validation Dataset'

# Create training dataset
train_ds = image_dataset_from_directory(
    train_dir,
    validation_split = 0,
    seed = 123,
    image_size = (img_height, img_width),
    batch_size = batch_size,
    label_mode = 'categorical'
)
# Normalize training dataset pixel values to improve learning performance
train_ds = train_ds.map(process)
# Prefetch training dataset to improve computational speed
train_ds = train_ds.cache().prefetch(buffer_size = autotune)

# Define validation dataset
val_ds = image_dataset_from_directory(
    validation_dir,
    validation_split = 0,
    seed = 123,
    image_size = (img_height, img_width),
    batch_size = batch_size,
    label_mode = 'categorical'
)
# Normalize validation pixel values to improve learning performance
val_ds = val_ds.map(process)
# Prefetch validation dataset to improve computation speed
val_ds = val_ds.cache().prefetch(buffer_size = autotune)

# Compile model:
  # Adam algorithm optimization
  # Categorical crossentropy loss function for multilabel classification
  # Metrics: accuracy, precision, recall
mobile_net_v2.compile(
    optimizer = 'adam',
    loss = 'categorical_crossentropy',
    metrics = ['accuracy', Precision(), Recall()]
)

# Define checkpoint to save model after every epoch
model_checkpoint = ModelCheckpoint('/content/drive/MyDrive/Colab Notebooks/Model Training and Evaluation/Models/MobileNetV2/Original Training/mobilenetv2-original-{epoch:02d}-{val_accuracy:.4f}.hdf5', save_best_only = False, save_weights_only = False, mode= 'auto')

# Train model
mobile_net_v2.fit(
    train_ds,
    validation_data = val_ds,
    epochs = epochs,
    verbose = 1,
    callbacks = [model_checkpoint]
)

9420800/9406464 [==============================] - 0s 0us/step
Found 2433 files belonging to 8 classes.
Found 695 files belonging to 8 classes.
Epoch 1/100
609/609 [==============================] - 41s 51ms/step - loss: 0.7818 - accuracy: 0.7226 - precision_1: 0.8348 - recall_1: 0.6293 - val_loss: 0.5323 - val_accuracy: 0.8014 - val_precision_1: 0.8363 - val_recall_1: 0.7496
Epoch 2/100
609/609 [==============================] - 7s 11ms/step - loss: 0.3713 - accuracy: 0.8627 - precision_1: 0.9016 - recall_1: 0.8323 - val_loss: 0.4723 - val_accuracy: 0.8302 - val_precision_1: 0.8571 - val_recall_1: 0.7942
Epoch 3/100
609/609 [==============================] - 7s 11ms/step - loss: 0.2563 - accuracy: 0.9133 - precision_1: 0.9396 - recall_1: 0.8948 - val_loss: 0.4561 - val_accuracy: 0.8432 - val_precision_1: 0.8666 - val_recall_1: 0.8129
Epoch 4/100
609/609 [==============================] - 7s 11ms/step - loss: 0.1886 - accuracy: 0.9482 - precision_1: 0.9641 - recall_1: 0.9268 - val_loss

*Augmented Dataset*

In [3]:
# Import os library to navigate directories
import os
# Import tensorflow functions for model building
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.layers import Dense, Input, GlobalAveragePooling2D
from tensorflow.keras import Model
from tensorflow.keras.models import save_model
# Import tensorflow functions for training
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.data import AUTOTUNE as autotune
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow import cast
# Import tensorflow functions for metrics
from tensorflow.keras.metrics import Accuracy, Precision, Recall

# ---------------------------- Build MobileNetV2 --------------------------- #
# Create base MobileNetV2 model:
  # 224x224x3 input size
  # Only feature extraction layers - no classification layers
  # Feature extraction layer weights trained on ImageNet dataset
base_model = MobileNetV2(input_shape = (224, 224, 3),
                         include_top = False,
                         weights = 'imagenet'
                         )
# Freeze learning on feature extraction layers
base_model.trainable = False

# Create classification layers of InceptionV3
global_average_pool = GlobalAveragePooling2D()        # global average pooling
classifier = Dense(units = 8, activation = 'softmax') # 1000 neuron fully connected layer with ReLU activation

# Connect input layer, base model and output layers
inputs = Input(shape = (224, 224, 3)) # Input layer: 224x224 RGB images
model = base_model(inputs)            # Feature extraction layers of InceptionV3
model = global_average_pool(model)    # Global average pooling layer
outputs = classifier(model)           # Fully connected layer: 8 neuron softmax activation for classification

# Build inception_v3 model
mobile_net_v2 = Model(inputs, outputs, name = 'MobileNetV2')

# ---------------------------- Train MobileNetV2 -------------------------- #
# RGB value normalization to improve learning rate - RGB values within activation function range
def process(image, label):
    image = cast(image/255.0, 'float32')
    return image, label

# Define training parameters
batch_size = 4   # Batch size for gradient learning
img_height = 224 # Image pixel height
img_width = 224  # Image pixel width
epochs = 100     # 100 rounds of training

# Define training and validation directory paths
train_dir = '/content/drive/MyDrive/Colab Notebooks/Model Training and Evaluation/Augmented Training Dataset'
validation_dir = '/content/drive/MyDrive/Colab Notebooks/Model Training and Evaluation/Validation Dataset'

# Create training dataset
train_ds = image_dataset_from_directory(
    train_dir,
    validation_split = 0,
    seed = 123,
    image_size = (img_height, img_width),
    batch_size = batch_size,
    label_mode = 'categorical'
)
# Normalize training dataset pixel values to improve learning performance
train_ds = train_ds.map(process)
# Prefetch training dataset to improve computational speed
train_ds = train_ds.cache().prefetch(buffer_size = autotune)

# Define validation dataset
val_ds = image_dataset_from_directory(
    validation_dir,
    validation_split = 0,
    seed = 123,
    image_size = (img_height, img_width),
    batch_size = batch_size,
    label_mode = 'categorical'
)
# Normalize validation pixel values to improve learning performance
val_ds = val_ds.map(process)
# Prefetch validation dataset to improve computation speed
val_ds = val_ds.cache().prefetch(buffer_size = autotune)

# Compile model:
  # Adam algorithm optimization
  # Categorical crossentropy loss function for multilabel classification
  # Metrics: accuracy, precision, recall
mobile_net_v2.compile(
    optimizer = 'adam',
    loss = 'categorical_crossentropy',
    metrics = ['accuracy', Precision(), Recall()]
)

# Define checkpoint to save model after every epoch
model_checkpoint = ModelCheckpoint('/content/drive/MyDrive/Colab Notebooks/Model Training and Evaluation/Models/MobileNetV2/Augmented Training/mobilenetv2-augmented-{epoch:02d}-{val_accuracy:.4f}.hdf5', save_best_only = False, save_weights_only = False, mode= 'auto')

# Train model
mobile_net_v2.fit(
    train_ds,
    validation_data = val_ds,
    epochs = epochs,
    verbose = 1,
    callbacks = [model_checkpoint]
)

Found 10083 files belonging to 8 classes.
Found 695 files belonging to 8 classes.
Epoch 1/100
2521/2521 [==============================] - 30s 11ms/step - loss: 0.6594 - accuracy: 0.7627 - precision_2: 0.8352 - recall_2: 0.7013 - val_loss: 0.4934 - val_accuracy: 0.8245 - val_precision_2: 0.8582 - val_recall_2: 0.8101
Epoch 2/100
2521/2521 [==============================] - 23s 9ms/step - loss: 0.3912 - accuracy: 0.8573 - precision_2: 0.8865 - recall_2: 0.8279 - val_loss: 0.5068 - val_accuracy: 0.8288 - val_precision_2: 0.8473 - val_recall_2: 0.8144
Epoch 3/100
2521/2521 [==============================] - 23s 9ms/step - loss: 0.3136 - accuracy: 0.8876 - precision_2: 0.9073 - recall_2: 0.8661 - val_loss: 0.5359 - val_accuracy: 0.8302 - val_precision_2: 0.8434 - val_recall_2: 0.8216
Epoch 4/100
2521/2521 [==============================] - 23s 9ms/step - loss: 0.2672 - accuracy: 0.9028 - precision_2: 0.9199 - recall_2: 0.8874 - val_loss: 0.5681 - val_accuracy: 0.8360 - val_precision_2: 0.8

# Model Testing

In [1]:
# Import directory navigation
from os import listdir
# Import tensorflow libraries for loading models and testing dataset
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow import cast
# Import tensorflow functions for metrics
from tensorflow.keras.metrics import Accuracy, Precision, Recall
#from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import numpy as np

# Import tensorflow functions for model building
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Dense, Input, GlobalAveragePooling2D
from tensorflow.keras import Model

# Constants for model file paths
base_model_path = '/content/drive/MyDrive/Colab Notebooks/Model Training and Evaluation/Models/'
models = ['VGG-16', 'ResNet-50', 'DenseNet121', 'InceptionV3', 'MobileNetV2']
# Empty array for best performing model in training groups
best_models = []

# ------------------------- Dataset and Testing Functions ------------------------ #
def process(image, label):
    image = cast(image/255.0, 'float32')
    return image, label

def test_models(test_ds, model_path):
    # Track best accuracy and model
    best_accuracy = 0
    best_model = ''
    # Get directory of models
    directory = listdir(model_path)
    # Loop through models and test
    for file in directory:
        print('Model: '+ file)
        model = load_model(model_path+'/'+file)
        loss, accuracy, precision, recall = model.evaluate(test_ds)
        print('Accuracy: ' + str(accuracy))
        print('Precision: ' + str(precision))
        print('Recall: ' + str(recall)+'\n')
        if (accuracy > best_accuracy):
            best_accuracy = accuracy
            best_model = model_path+'/'+file
    return best_model

# ----------------------------- Load Testing Dataset ----------------------------- #
# Define testing dataset
test_dir = '/content/drive/MyDrive/Colab Notebooks/Model Training and Evaluation/Testing Dataset'
test_ds = image_dataset_from_directory(
    test_dir,
    validation_split = 0,
    seed = 123,
    image_size = (224, 224),
    batch_size = 346,
    label_mode = 'categorical'
)
# Normalize testing pixel values
test_ds = test_ds.map(process)

# --------------------------------- Test Models ---------------------------------- #
for model in models:
    print('\n# ---------------------------------------- '+model+' ---------------------------------------- #\n')
    # Original dataset training performance
    print('{}{}{}'.format('\033[1m', 'Original Dataset Training:\n', '\033[0m'))
    best_models.append(test_models(test_ds, base_model_path+model+'/Original Training'))
    # Augmented dataset training perforamnce
    print('{}{}{}'.format('\033[1m', 'Augmented Dataset Training:\n', '\033[0m'))
    best_models.append(test_models(test_ds, base_model_path+model+'/Augmented Training'))

# ----------------------------- Compare Best Models ------------------------------ #
print('\n# -------------------------------------- Best Performers -------------------------------------- #\n')
for model in best_models:
    print(model)
    model = load_model(model)
    loss, accuracy, precision, recall = model.evaluate(test_ds)
    print('Accuracy: ' + str(accuracy))
    print('Precision: ' + str(precision))
    print('Recall: ' + str(recall)+'\n')

Found 346 files belonging to 8 classes.

# ---------------------------------------- VGG-16 ---------------------------------------- #

Original Dataset Training:

Model: vgg16-original-01-0.6173.hdf5
1/1 [==============================] - 24s 24s/step - loss: 0.9436 - accuracy: 0.6185 - precision_10: 0.8068 - recall_10: 0.4827
Accuracy: 0.6184971332550049
Precision: 0.8067632913589478
Recall: 0.4826589524745941

Model: vgg16-original-02-0.6014.hdf5
1/1 [==============================] - 1s 1s/step - loss: 1.0038 - accuracy: 0.5694 - precision_10: 0.6603 - recall_10: 0.5000
Accuracy: 0.5693641901016235
Precision: 0.6603053212165833
Recall: 0.5

Model: vgg16-original-03-0.6518.hdf5
1/1 [==============================] - 1s 1s/step - loss: 0.9272 - accuracy: 0.6127 - precision_10: 0.7019 - recall_10: 0.5376
Accuracy: 0.6127167344093323
Precision: 0.701886773109436
Recall: 0.5375722646713257

Model: vgg16-original-04-0.7036.hdf5
1/1 [==============================] - 1s 949ms/step - loss: 

1/1 [==============================] - 1s 930ms/step - loss: 0.8491 - accuracy: 0.7168 - precision_10: 0.7682 - recall_10: 0.6705
Accuracy: 0.7167630195617676
Precision: 0.7682119011878967
Recall: 0.6705202460289001

Model: vgg16-original-06-0.7266.hdf5


Streaming output truncated to the last 5000 lines.

Model: vgg16-augmented-83-0.8072.hdf5
1/1 [==============================] - 1s 954ms/step - loss: 3.5273 - accuracy: 0.7861 - precision_11: 0.7884 - recall_11: 0.7861
Accuracy: 0.7861271500587463
Precision: 0.7884057760238647
Recall: 0.7861271500587463

Model: vgg16-augmented-84-0.8086.hdf5
1/1 [==============================] - 1s 957ms/step - loss: 2.9818 - accuracy: 0.7919 - precision_11: 0.7959 - recall_11: 0.7890
Accuracy: 0.7919074892997742
Precision: 0.795918345451355
Recall: 0.7890173196792603

Model: vgg16-augmented-85-0.7928.hdf5
1/1 [==============================] - 1s 946ms/step - loss: 2.6995 - accuracy: 0.7832 - precision_11: 0.7832 - recall_11: 0.7832
Accuracy: 0.7832369804382324
Precision: 0.7832369804382324
Recall: 0.7832369804382324

Model: vgg16-augmented-86-0.7885.hdf5
1/1 [==============================] - 1s 963ms/step - loss: 3.2923 - accuracy: 0.7948 - precision_11: 0.7994 - recall_11: 0.7948
Accuracy: 0.7947

# Data Augmentation and Preprocessing

In [ ]:
!pip install Augmentor

*Dataset Split and Preprocessing: 70:20:10 (training, validation, testing) split*

In [ ]:
# Import directory navigation libraries and image resizing
from os import listdir
from shutil import move
from random import randint
from PIL import Image
from tensorflow.image import resize
from tensorflow.io import decode_jpeg, encode_jpeg, write_file
from tensorflow.dtypes import saturate_cast

# Constant values
base_path = '/content/drive/MyDrive/Colab Notebooks/Model Training and Evaluation/'
base_validation_path = '/content/drive/MyDrive/Colab Notebooks/Model Training and Evaluation/Validation Dataset/'
base_test_path = '/content/drive/MyDrive/Colab Notebooks/Model Training and Evaluation/Testing Dataset/'
base_original_training_path = '/content/drive/MyDrive/Colab Notebooks/Model Training and Evaluation/Original Training Dataset/'
labels = ['Glass', 'Plastic', 'Paper', 'Metal', 'Cardboard', 'Vegetation', 'Food', 'Trash']

# ------------------------------- Functions -------------------------------- #
# Resize images
def resize_image(path, img_width, img_height):
    directory = listdir(path)
    for file in directory:
        image = open(path+'/'+file, 'rb') # Open image as binary file
        binary_representation = image.read() # Read binary file
        decoded_representation = decode_jpeg(binary_representation) # Decode image
        resized_image = resize(decoded_representation, [img_width, img_height])
        resized_image = saturate_cast(resized_image, 'uint8')
        encoded_image = encode_jpeg(resized_image)
        write_file(path+'/'+file, encoded_image)

# Check images for RGB mode and convert if required
def convert_rgb_images(path):
    directory = listdir(path)
    for file in directory:
        img = Image.open(path+'/'+file)
        if img.mode != 'RGB':
            img.convert('RGB').save(path+'/'+file)
        
# Count number of files in directory
def count_files(inputPath, label):
    directory = listdir(inputPath+label)
    num_files = 0
    for file in directory:
        num_files += 1
    return num_files

# Split training and testing dataset
def split_dataset(input_path, train_path, val_path, test_path, val_prop, test_prop, label):
    # Count images in directory and get files names
    num_files = count_files(input_path, label)
    file_names = listdir(input_path+label)

    # Calculated number of validation and testing images
    num_val = round(num_files*val_prop)
    num_test = round(num_files*test_prop)
    
    # Initialise empty lists for validation and testing image indexes
    val_list = []
    test_list = []
    
    # Randomly assign file indexes for validation dataset
    i = 0
    while i < num_val:
        r = randint(1, num_files - 1)
        if r not in val_list:
            val_list.append(r)
            i += 1
    # Move validation dataset files and rename
    name_incrementer = 1
    for j in val_list:
        move(input_path+label+'/'+file_names[j], val_path+label+'/'+label+'_'+str(name_incrementer)+'.jpg')
        name_incrementer += 1

    # Update number of files and directory for remaining files
    num_files = count_files(input_path, label)
    remaining_files = listdir(input_path+label)
    # Randomly assign file indexes for testing dataset
    i = 0
    while i < num_test:
        r = randint(1, num_files - 1)
        if r not in test_list:
            test_list.append(r)
            i += 1
    # Move test dataset files and rename
    name_incrementer = 1
    for j in test_list:
        move(input_path+label+'/'+remaining_files[j], test_path+label+'/'+label+'_'+str(name_incrementer)+'.jpg')
        name_incrementer += 1

    # Move remaining files into training dataset
    training_files = listdir(input_path+label)
    name_incrementer = 1
    for file in training_files:
        move(input_path+label+'/'+file, train_path+label+'/'+label+'_'+str(name_incrementer)+'.jpg')
        name_incrementer += 1

# Print number of files in label directories
def print_num_files(path, label):
    directory = listdir(path+label)
    counter = 0
    for file in directory:
        counter += 1
    print("Number of files in " + label +':' + str(counter))

# ----------------- Split Training and Validation Datasets ----------------- #
# Print split label contents
print('# --------------- Total Label Counts ---------------- #')
# Split labels base off 70:20:10 training split
for label in labels:
    print_num_files(base_path+'Unsplit Dataset/', label)
    split_dataset(base_path+'Unsplit Dataset/', base_original_training_path, base_validation_path, base_test_path, 0.2, 0.1, label)

# Print validation counts for each label
print('\n# ---------- Total Validation Label Counts ---------- #')
for label in labels:
    print_num_files(base_validation_path, label)

# Print testing counts for each label
print('\n# ------------ Total Testing Label Counts ----------- #')
for label in labels:
    print_num_files(base_test_path, label)

# Print training counts for each label in training dataset
print('\n# ----------- Total Training Label Counts ----------- #')
for label in labels:
    print_num_files(base_original_training_path, label)

# ------------------------- Pre-Process Images -------------------------- #
# Pre-process training dataset
print('\n# --------- Pre-Processing Training Dataset --------- #')
for label in labels:
    convert_rgb_images(base_original_training_path+label)
    resize_image(base_original_training_path+label, 224, 224)
# Pre-process validation dataset
print('\n# -------- Pre-Processing Validation Dataset -------- #')
for label in labels:
    convert_rgb_images(base_original_training_path+label)
    resize_image(base_validation_path+label, 224, 224)
# Pre-process test dataset
print('\n# --------- Pre-Processing Testing Dataset --------- #')
for label in labels:
    convert_rgb_images(base_original_training_path+label)
    resize_image(base_test_path+label, 224, 224)

*Augementation Set 1: Distort and Flip*

In [ ]:
# Import data augmentation functions
import Augmentor
from os import listdir
from time import sleep

# Define base path of training dataset
base_original_training_path = '/content/drive/MyDrive/Colab Notebooks/Model Training and Evaluation/Original Training Dataset/'
# Define labels
#labels = ['Glass', 'Plastic', 'Paper', 'Metal', 'Cardboard', 'Vegetation', 'Food', 'Trash']
labels = ['Glass']

# Verify augmentation counts and print
def count_augmented_and_print(path):
    count_augmented = 0
    directory = listdir(path)
    for file in directory:
        count_augmented += 1
    print('Processed with '+str(count_augmented)+' image(s) found.\n')

# Geometric augmentation function: distort and flip
def distort_and_flip(path):
    pipe = Augmentor.Pipeline(path)           # Create pipeline in path of dataset
    pipe.flip_left_right(1)                   # Horizontally flip every image
    pipe.random_distortion(1, 5, 5, 4)        # Randomly distort flipped images
    pipe.process()                            # Process images in pipeline
    count_augmented_and_print(path+'/output') # Print augmented image count to console
    sleep(60)                                 # Sleep program while images processing

# Perform augmentation
for label in labels:
    distort_and_flip(base_original_training_path+label)

*Augmentation Set 2: Rotate and Shear*

In [ ]:
# Import data augmentation functions
import Augmentor
from os import listdir
from time import sleep

# Define base path of training dataset
base_original_training_path = '/content/drive/MyDrive/Colab Notebooks/Model Training and Evaluation/Original Training Dataset/'
# Define labels
labels = ['Glass', 'Plastic', 'Paper', 'Metal', 'Cardboard', 'Vegetation', 'Food', 'Trash']

# Verify augmentation counts and print
def count_augmented_and_print(path):
    count_augmented = 0
    directory = listdir(path)
    for file in directory:
        count_augmented += 1
    print('Processed with '+str(count_augmented)+' image(s) found.\n')

# Geometric augmentation: rotate and shear
def rotate_and_shear(path):
    pipe = Augmentor.Pipeline(path)           # Create pipeline in path of dataset
    pipe.rotate(1, 25, 25)                    # Rotate every image by 45 degrees
    pipe.shear(1, 15, 15)                     # Randomly shear between 0 and 15 degrees
    pipe.process()                            # Process images in pipeline
    count_augmented_and_print(path+'/output') # Print augmented image count to console
    sleep(60)                                 # Sleep program while images processing

# Augment images
for label in labels:
    rotate_and_shear(base_original_training_path+label)

*Augmentation Set 3: Random Erasing*

In [ ]:
# Import data augmentation functions
import Augmentor
from os import listdir
from time import sleep

# Define base path of training dataset
base_original_training_path = '/content/drive/MyDrive/Colab Notebooks/Model Training and Evaluation/Original Training Dataset/'
# Define labels
labels = ['Glass', 'Plastic', 'Paper', 'Metal', 'Cardboard', 'Vegetation', 'Food', 'Trash']

# Verify augmentation counts and print
def count_augmented_and_print(path):
    count_augmented = 0
    directory = listdir(path)
    for file in directory:
        count_augmented += 1
    print('Processed with '+str(count_augmented)+' image(s) found.\n')

# Random occlusion: random erasing 
def random_erase(path):
    pipe = Augmentor.Pipeline(path)           # Create pipeline in path of dataset
    pipe.random_erasing(1, 0.15)              # Randomly erase area in every image
    pipe.process()                            # Process images in pipeline
    count_augmented_and_print(path+'/output') # Print augmented image count to console
    sleep(60)                                 # Sleep program while images processing

# Augment images
for label in labels:
    random_erase(base_original_training_path+label)

# Second augmentation
for label in labels:
    random_erase(base_original_training_path+label+'/output')

*Rename Files*

In [ ]:
# Import libraries
from os import listdir
from shutil import move

# Constants
set_one_path = '/content/drive/MyDrive/Colab Notebooks/Model Training and Evaluation/1 Distort and Flip/'
set_two_path = '/content/drive/MyDrive/Colab Notebooks/Model Training and Evaluation/2 Rotate and Shear/'
set_three_path = '/content/drive/MyDrive/Colab Notebooks/Model Training and Evaluation/3 Random Erasing/'
set_one_augmentation = 'distort_and_flip'
set_two_augmentation = 'shear_and_rotate'
set_three_augmentation = 'random_erase'
labels = ['Glass', 'Plastic', 'Paper', 'Metal', 'Cardboard', 'Vegetation', 'Food', 'Trash']

# Get files in directory and rename
def rename_files(path, base_name, augmentation):
    name_incrementer = 1
    directory = listdir(path)
    for file in directory:
        move(path+'/'+file, path+'/'+base_name+'_'+augmentation+'_'+str(name_incrementer)+'.jpg')
        name_incrementer += 1
        print(file)
    return name_incrementer

# Rename files
for label in labels:
    rename_files(set_one_path+label+'/output', label, set_one_augmentation)
    rename_files(set_two_path+label+'/output', label, set_two_augmentation)
    rename_files(set_three_path+label+'/output', label, set_three_augmentation)

*File Counts*

In [ ]:
# Import file navigation library
from os import listdir

# Directory paths
base_validation_path = '/content/drive/MyDrive/Colab Notebooks/Model Training and Evaluation/Validation Dataset/'
base_test_path = '/content/drive/MyDrive/Colab Notebooks/Model Training and Evaluation/Testing Dataset/'
base_original_training_path = '/content/drive/MyDrive/Colab Notebooks/Model Training and Evaluation/Original Training Dataset/'
base_augmented_training_path = '/content/drive/MyDrive/Colab Notebooks/Model Training and Evaluation/Augmented Training Dataset/'

# Print number of files in label directories
def print_num_files(path, label):
    directory = listdir(path+label)
    counter = 0
    for file in directory:
        counter += 1
    print("Number of files in " + label +':' + str(counter))

# Print training counts for each label in training dataset
print('\n# ----------- Total Training Label Counts ----------- #')
for label in labels:
    print_num_files(base_original_training_path, label)

# Print validation counts for each label
print('\n# ---------- Total Validation Label Counts ---------- #')
for label in labels:
    print_num_files(base_validation_path, label)

# Print testing counts for each label
print('\n# ------------ Total Testing Label Counts ----------- #')
for label in labels:
    print_num_files(base_test_path, label)

# Print training counts for each label in training dataset
print('\n# ----------- Total Training Label Counts ----------- #')
for label in labels:
    print_num_files(base_original_training_path, label)

# Print augmented counts for each label in augmented training dataset
print('\n# ----------- Total Augmented Label Counts ----------- #')
for label in labels:
    print_num_files(base_original_training_path, label)

In [ ]:
# Import data augmentation functions
import Augmentor

# Define path of sample image for each augmentation
base_path_set_one = '/content/drive/MyDrive/Colab Notebooks/Data Augmentation Samples/1 Distort and Flip'
base_path_set_two = '/content/drive/MyDrive/Colab Notebooks/Data Augmentation Samples/2 Rotate and Shear'
base_path_set_three = '/content/drive/MyDrive/Colab Notebooks/Data Augmentation Samples/3 Random Erasing'

# Geometric augmentation function: distort and flip
def distort_and_flip(path):
    pipe = Augmentor.Pipeline(path)           # Create pipeline in path of dataset
    pipe.flip_left_right(1)                   # Horizontally flip every image
    pipe.random_distortion(1, 5, 5, 4)        # Randomly distort flipped images
    pipe.process()                            # Process images in pipeline

# Geometric augmentation: rotate and shear
def rotate_and_shear(path):
    pipe = Augmentor.Pipeline(path)           # Create pipeline in path of dataset
    pipe.rotate(1, 25, 25)                    # Rotate every image by 45 degrees
    pipe.shear(1, 15, 15)                     # Randomly shear between 0 and 15 degrees
    pipe.process()                            # Process images in pipeline

# Random occlusion: random erasing 
def random_erase(path):
    pipe = Augmentor.Pipeline(path)           # Create pipeline in path of dataset
    pipe.random_erasing(1, 0.15)              # Randomly erase area in every image
    pipe.process()                            # Process images in pipeline

# Perform augmentations
# Distort and flip
distort_and_flip(base_path_set_one)
# Rotate and shear
rotate_and_shear(base_path_set_two)
# Random erase
random_erase(base_path_set_three)
random_erase(base_path_set_three+'/output')

# Training Results Processing

In [ ]:
# Import file navigation libraries and file writing
from os import listdir
import csv

# Constants for model file paths
base_model_path = '/content/drive/MyDrive/Colab Notebooks/Model Training and Evaluation/Models/'
raw_performance_directory = '/Performance/Raw/'
curated_performance_directory = '/Performance/Curated/'
models = ['VGG-16', 'ResNet-50', 'DenseNet121', 'InceptionV3', 'MobileNetV2']
# Constants for epochs to loop over
epochs = 200
# Field headers
field_headers = ['Epoch', 'TrainLoss', 'TrainAccuracy', 'TrainPrecision', 'TrainRecall', 'ValLoss', 'ValAccuracy', 'ValPrecision', 'ValRecall']

# Write to files
def write_files(base_path, raw_directory, curated_directory, model, epochs, fields):
    # Get files in directory
    directory = listdir(base_path+model+raw_directory)
    for file in directory:
        # Open raw txt file
        raw = open(base_path+model+raw_directory+file, 'r')
        # Parse csv file name
        curated_name = file.split('.')[0] + '.csv'
        # Open curated csv file
        with open(base_path+model+curated_directory+curated_name, 'r+') as csv_file:
            # Create csv writer instance
            csv_writer = csv.writer(csv_file)
            
            # Write header rows to csv
            csv_writer.writerow(fields)

            # Parse results from txt and write to csv
            i = 1
            counter = 1
            for line in raw:
                if line[0:5] != 'Epoch' and i in range(6, (epochs*2)+6):
                    # Parse out results from lines in txt file
                    line = line.split(' - ')
                    train_loss = line[2].split(': ')[1]
                    train_accuracy = line[3].split(': ')[1]
                    train_precision = line[4].split(': ')[1]
                    train_recall = line[5].split(': ')[1]
                    val_loss = line[6].split(': ')[1]
                    val_accuracy = line[7].split(': ')[1]
                    val_precision = line[8].split(': ')[1]
                    val_recall = line[9].split(': ')[1].replace('\n', '')
                    # Parse results into csv format and write line to file
                    line = [counter, train_loss, train_accuracy, train_precision, train_recall, val_loss, val_accuracy, val_precision, val_recall]
                    csv_writer.writerow(line)
                    counter += 1
                i += 1
        raw.close()
        print('Finished writing to file: '+curated_name)

# Loop through folders and parse results into csv
for model in models:
    write_files(base_model_path, raw_performance_directory, curated_performance_directory, model, epochs, field_headers)